# Developing `load_era5()` function from Google

Guidance from https://github.com/google-research/arco-era5/blob/main/docs/0-Surface-Reanalysis-Walkthrough.ipynb

In [1]:
import fsspec
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import numpy as np

fs = fsspec.filesystem('gs')
fs.ls('gs://gcp-public-data-arco-era5/co/')

['gcp-public-data-arco-era5/co/model-level-moisture.zarr',
 'gcp-public-data-arco-era5/co/model-level-moisture.zarr-v2',
 'gcp-public-data-arco-era5/co/model-level-wind.zarr',
 'gcp-public-data-arco-era5/co/model-level-wind.zarr-v2',
 'gcp-public-data-arco-era5/co/single-level-forecast.zarr',
 'gcp-public-data-arco-era5/co/single-level-forecast.zarr-v2',
 'gcp-public-data-arco-era5/co/single-level-reanalysis.zarr',
 'gcp-public-data-arco-era5/co/single-level-reanalysis.zarr-v2',
 'gcp-public-data-arco-era5/co/single-level-surface.zarr',
 'gcp-public-data-arco-era5/co/single-level-surface.zarr-v2']

## Function to convert longitude from 0 - 360 from -180 - 180

In [2]:
def lon_to_360(dlon: float) -> float:
    return ((360 + (dlon % 360)) % 360)

## Select variables from ERA5, this will have to be a dictionary lookup eventually

In [3]:
variables = 't2m' #temperature 2 metres (I think)

## Load reanalysis

In [4]:
time_min, time_max = ['2021-01-01', '2021-03-31']

In [5]:
reduce_func = None
resample="1D"

if variables == 'tp':
    url = 'gs://gcp-public-data-arco-era5/co/single-level-forecast.zarr-v2/'
else:
    url = 'gs://gcp-public-data-arco-era5/co/single-level-reanalysis.zarr-v2'

if reduce_func is None:
        reduce_func = np.mean
        
reanalysis = xr.open_zarr(
    url, 
    chunks={'time': 48},
    consolidated=True).sel(time=slice(time_min, time_max))[[variables]].compute()
if variables == 'tp':
    reanalysis = reanalysis.resample(step=resample).reduce(reduce_func)
    reanalysis = reanalysis.resample(time=resample).reduce(reduce_func)
else:
    reanalysis = reanalysis.resample(time=resample).reduce(reduce_func)

xmin, xmax, ymin, ymax = [5.8, 58, -37, 0]
reanalysis = reanalysis.where(
    (reanalysis.longitude > lon_to_360(xmin)) & (reanalysis.latitude > ymin) &
    (reanalysis.longitude < lon_to_360(xmax)) & (reanalysis.latitude < ymax),
    drop=True
)
reanalysis =  (reanalysis.
               to_dataframe().
                      droplevel(1).
                      reset_index().
                      set_index(['time','latitude','longitude']).
                      to_xarray())
reanalysis = reanalysis.rename({'latitude':'lat','longitude': 'lon'})
reanalysis

/usr/local/lib/python3.10/dist-packages/xarray/core/dataset.py:265: UserWarning: The specified chunks separate the stored chunks along dimension "time" starting at index 48. This could degrade performance. Instead, consider rechunking after loading.
  warnings.warn(


<xarray.Dataset>
Dimensions:              (time: 90, lat: 132, lon: 928)
Coordinates:
  * time                 (time) datetime64[ns] 2021-01-01 ... 2021-03-31
  * lat                  (lat) float64 -36.96 -36.67 -36.39 ... -0.4215 -0.1405
  * lon                  (lon) float64 5.906 5.926 5.938 ... 57.9 57.92 57.94
Data variables:
    t2m                  (time, lat, lon) float32 nan nan nan ... nan nan 301.1
    depthBelowLandLayer  (time, lat, lon) float64 nan nan nan ... nan nan 100.0
    entireAtmosphere     (time, lat, lon) float64 nan nan nan ... nan nan 0.0
    number               (time, lat, lon) float64 nan nan nan ... nan nan 0.0
    step                 (time, lat, lon) timedelta64[ns] NaT NaT ... 00:00:00
    surface              (time, lat, lon) float64 nan nan nan ... nan nan 0.0

## Next steps

1. Assign spatial dimensions and crs to xarray dataset
2. Wrap up in `load_era5()` function.